# Multidimensional Analysis

In this notebook, we'll be show you how to perform multidimensional analysis, allowing users to observe data from different viewpoints.

<div style="text-align: center;"><img src="https://data.atoti.io/notebooks/multidimension/img/cube-slice-dice.png"></div>

<div style="text-align:center"><a href="https://www.atoti.io/?utm_source=gallery&utm_content=multidimensional-analysis" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Discover-Atoti-now.png" alt="atoti table" /></a></div>

In [1]:
import pandas as pd
import atoti as tt

In [ ]:
session = tt.Session.start()

In [ ]:
sales = session.read_csv(
    "s3://data.atoti.io/notebooks/multidimension/sales.csv",
    table_name="Sales",
    data_types={"Unit price": tt.type.DOUBLE},
)
sales.head()

In [ ]:
cube = session.create_cube(sales, "sales cube")

# Data are group into two categories - Data dimensions and measurements

In [ ]:
cube

<img alt="Multidimensional cube concept" src="https://data.atoti.io/notebooks/multidimension/img/cube-concept.svg" width="400" />

The axes of the cube are called _hierarchies_.

In [ ]:
session.widget

# Capable of integrating multiple data sources 

In [ ]:
products = session.read_csv(
    "s3://data.atoti.io/notebooks/multidimension/products.csv",
    table_name="Products",
    keys=["Product"],
)
products.head()

In [ ]:
sales.join(products)
session.tables.schema

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [ ]:
h

In [ ]:
h["Product categories"] = [l["Category"], l["Sub category"]]

In [ ]:
session.widget

# Aggregational flexibility - Pre-processing vs Post-processing



## Pre-processing trades storage consumption for computational efficiency

In [ ]:
product_df = pd.read_csv("https://data.atoti.io/notebooks/multidimension/products.csv")
product_df.head()

In [ ]:
sales_df = pd.read_csv("https://data.atoti.io/notebooks/multidimension/sales.csv")
sales_df.head()

Within the same data source, we can easily perform aggregation by applying the formulas. Notice that our dataset just grew bigger with the additional column? By loading this dataset, we traded storage consumption for the computational efficiency because the cube need not compute the sales amount every time we query it.

In [ ]:
sales_df["Sales amount"] = sales_df["Quantity"] * sales_df["Unit price"]
sales_df.head()

When aggregation involves multiple data sources, then we need to handle the joins before we can apply the formulas.

In [ ]:
product_sales = pd.merge(sales_df, product_df, on="Product")
product_sales.head()

In [ ]:
product_sales["Cost price"] = (
    product_sales["Quantity"] * product_sales["Purchase price"]
)
product_sales["Margin"] = sales_df["Sales amount"] - product_sales["Cost price"]
product_sales.head()

Pre-processing works when we are at the most granular level because we will still be able to drill down or roll up. At higher aggregation level, we will not know how to break the value when we drill down to the lower level.

<div style="text-align: center;"><img src="https://data.atoti.io/notebooks/multidimension/img/drilldown_rollup.png" width="50%"></div>

## Post-processing computes on the fly and provides flexibility

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [ ]:
m["Sales amount"] = tt.agg.sum(
    sales["Quantity"] * sales["Unit price"],
)

In [ ]:
m["Cost price"] = tt.agg.sum(
    m["Quantity.SUM"] * tt.agg.single_value(products["Purchase price"]),
    scope=tt.OriginScope(levels={l["Product"]}),
)

In [ ]:
m["Margin"] = m["Sales amount"] - m["Cost price"]

In [ ]:
m["Cumulative amount"] = tt.agg.sum(
    m["Sales amount"], scope=tt.CumulativeScope(level=l["Date"])
)

m["Cumulative margin"] = tt.agg.sum(
    m["Margin"], scope=tt.CumulativeScope(level=l["Date"])
)

In the visualization below, we are missing the sales data for `Shop_0` on 15 Jan 2021. Suppose we use pre-processing computation to get the cumulative values, we will need to perform re-calculation and then reload the data into the cube. With post-processing, this is not required as the data is computed on the fly.

In [ ]:
session.widget

In [ ]:
session.widget

Simply load the missing data into the cube and it will be computed automatically.

In [ ]:
sales.load(tt.CsvLoad("s3://data.atoti.io/notebooks/multidimension/shop0_20210115.csv"))

Re-running the last 2 visualizations should reflect the correct sales.

<div style="text-align:center"><a href="https://www.atoti.io/?utm_source=gallery&utm_content=multidimensional-analysis" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Your-turn-to-try-Atoti.jpg" alt="Atoti table" /></a></div>